In [19]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = ""
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [69]:
class AgentReACT():
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role":"system", "content": self.system})

    def __call__(self, message=""):
        """This will be called on instance of class"""
        if message:
            self.messages.append({"role": "user", "content":message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content":result})
        return result

    def execute(self):
        chat_completion = client.chat.completions.create(
            messages=self.messages,
            model="llama3-8b-8192",
        )
        return chat_completion.choices[0].message.content

In [79]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
ALWAYS Remeber ONLY ONE Action & Argument ONCE at a time
Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

get_weight_of_person:
e.g. get_weight_of_person: Simmo
returns weight of person in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

def calculate(operation: str) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    if planet.lower() == "earth":
        return 5.972e24
    if planet.lower() == "mars":
        return 6.39e23
    if planet.lower() == "jupiter":
        return 1.898e27
    if planet.lower() == "saturn":
        return 5.683e26
    if planet.lower() == "uranus":
        return 8.681e25
    if planet.lower() == "neptune":
        return 1.024e26
    if planet.lower() == "mercury":
        return 3.285e23
    if planet.lower() == "venus":
        return 4.867e24
    return None

def get_weight_of_person(person_name):
    print("get_weight_of_person called with: ",person_name)
    match person_name.lower():
        case "jimmy":
            return 78
        case "bablu":
            return 90
        case "simmo":
            return 69

In [33]:
agent = AgentReACT(client, system_prompt)

In [34]:
agent("what is the mass of earth times 2")

'Thought: I need to find the mass of Earth\n\nAction: get_planet_mass: Earth\nPAUSE'

In [35]:
agent.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [41]:
calculate("5.972e24*2")

1.1944e+25

In [42]:
agent("Observation: 1.1944e+25")

'Answer: The mass of Earth times 2 is 1.1944e+25.'

In [43]:
agent.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [54]:
import re

In [77]:
def execuate_agent(query):
    agent = AgentReACT(client, system_prompt)
    tools = ["calculate", "get_planet_mass", "get_weight_of_person"]
    next_prompt = query
    while True:
        result = agent(next_prompt)
        print(result)
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            print("next_prompt: ", next_prompt)
            continue
        
        if "Answer" in result:
            break
    for i in agent.messages:
        print(i)

In [81]:
execuate_agent("what is the weight of tahir, divided by 3")

Thought: I need to find the weight of Tahir and then divide it by 3.

Action: get_weight_of_person: Tahir

PAUSE
get_weight_of_person called with:  Tahir
next_prompt:  Observation: 78
Thought: Now I need to divide the weight of Tahir by 3.

Action: calculate: 78 / 3

PAUSE
next_prompt:  Observation: 26.0
Thought: I think I have the answer. The weight of Tahir divided by 3 is 26.0.

Answer: The weight of Tahir divided by 3 is 26.0.
{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\nALWAYS Remeber ONLY ONE Action & Argument ONCE at a time\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point 

In [76]:
for i in agent.messages:
    print(i.replace())

{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the an